## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [33]:
import os
import mysql.connector
from dotenv import load_dotenv


load_dotenv()


cnx = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
print(cnx)


Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [2]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [3]:
cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")

In [35]:
cursor.execute("SHOW DATABASES")

for db in cursor:
    print(db)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [5]:
import pandas as pd


df = pd.read_csv("../data/books_data.csv")
df.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,678fc8ec5028200ccb8bf14b,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,678fc8ec5028200ccb8bf14c,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,678fc8ec5028200ccb8bf15e,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,678fc8ec5028200ccb8bf16c,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,678fc8ec5028200ccb8bf16e,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _id                     742 non-null    object 
 1   Produto                 742 non-null    object 
 2   Categoria do Produto    742 non-null    object 
 3   Preço                   742 non-null    float64
 4   Frete                   742 non-null    float64
 5   Data da Compra          742 non-null    object 
 6   Vendedor                742 non-null    object 
 7   Local da compra         742 non-null    object 
 8   Avaliação da compra     742 non-null    int64  
 9   Tipo de pagamento       742 non-null    object 
 10  Quantidade de parcelas  742 non-null    int64  
 11  Latitude                742 non-null    float64
 12  Longitude               742 non-null    float64
dtypes: float64(4), int64(2), object(7)
memory usage: 75.5+ KB


In [7]:
df.shape

(742, 13)

In [8]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
        id VARCHAR(100),
        produto VARCHAR(100),
        categoria VARCHAR(100),
        preco FLOAT(10,2),
        frete FLOAT(10,2),
        data_compra DATE,
        vendedor VARCHAR(200),
        local VARCHAR(100),
        avaliacao INT,
        tipo_pagamento VARCHAR(100),
        qtd_parcelas INT,
        latitude FLOAT(10,2),
        longitude FLOAT(10,2),

        PRIMARY KEY (id)
    );
    """
)

**Selecionando a base de dados para verificar a tabela criada**

In [9]:
cursor.execute("USE dbprodutos")
cursor.execute("SHOW TABLES")

for tb in cursor:
    print(tb)

('tb_livros',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [11]:
book_list = [tuple(row) for i, row in df.iterrows()]
book_list

[('678fc8ec5028200ccb8bf14b',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('678fc8ec5028200ccb8bf14c',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('678fc8ec5028200ccb8bf15e',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('678fc8ec5028200ccb8bf16c',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('678fc8ec5028200ccb8bf16e',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('678fc8ec5028200ccb8bf171',
  'Iniciando em progr

In [14]:
sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

In [15]:
cursor.executemany(sql, book_list)
cnx.commit()

In [16]:
print(cursor.rowcount, "dados foram inseridos.")

742 dados foram inseridos.


In [17]:
df_products_2021_onwards = pd.read_csv('../data/products_from_2021_onwards.csv')
df_products_2021_onwards.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,678fc8ec5028200ccb8bf154,Xadrez de madeira,brinquedos,25.23,0.000000,2021-01-01,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
1,678fc8ec5028200ccb8bf155,Impressora,eletronicos,322.04,14.732100,2021-01-01,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
2,678fc8ec5028200ccb8bf156,Mesa de centro,moveis,282.22,12.611805,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
3,678fc8ec5028200ccb8bf157,Tablet ABXY,eletronicos,1100.42,68.451348,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
4,678fc8ec5028200ccb8bf158,Fogão,eletrodomesticos,791.81,42.444626,2021-01-01,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


In [18]:
products_2021_onwards_list = [tuple(row) for i, row in df_products_2021_onwards.iterrows()]
products_2021_onwards_list[:5]

[('678fc8ec5028200ccb8bf154',
  'Xadrez de madeira',
  'brinquedos',
  25.23,
  0.0,
  '2021-01-01',
  'Thiago Silva',
  'BA',
  5,
  'cartao_credito',
  2,
  -13.29,
  -41.71),
 ('678fc8ec5028200ccb8bf155',
  'Impressora',
  'eletronicos',
  322.04,
  14.7321002401,
  '2021-01-01',
  'João Souza',
  'SP',
  3,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('678fc8ec5028200ccb8bf156',
  'Mesa de centro',
  'moveis',
  282.22,
  12.6118050347,
  '2021-01-01',
  'João Souza',
  'SP',
  3,
  'boleto',
  1,
  -22.19,
  -48.79),
 ('678fc8ec5028200ccb8bf157',
  'Tablet ABXY',
  'eletronicos',
  1100.42,
  68.4513476183,
  '2021-01-01',
  'Beatriz Moraes',
  'SP',
  1,
  'cupom',
  1,
  -22.19,
  -48.79),
 ('678fc8ec5028200ccb8bf158',
  'Fogão',
  'eletrodomesticos',
  791.81,
  42.4446263246,
  '2021-01-01',
  'Juliana Costa',
  'SP',
  4,
  'boleto',
  1,
  -22.19,
  -48.79)]

In [25]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_produtos_2021_em_diante(
        id VARCHAR(100),
        produto VARCHAR(100),
        categoria VARCHAR(100),
        preco FLOAT(10,2),
        frete FLOAT(10,2),
        data_compra DATE,
        vendedor VARCHAR(200),
        local VARCHAR(100),
        avaliacao INT,
        tipo_pagamento VARCHAR(100),
        qtd_parcelas INT,
        latitude FLOAT(10,2),
        longitude FLOAT(10,2),

        PRIMARY KEY (id)
    );
    """
)

In [26]:
cursor.execute("USE dbprodutos")
cursor.execute("SHOW TABLES")

for tb in cursor:
    print(tb)

('tb_livros',)
('tb_produtos_2021_em_diante',)


In [27]:
sql = "INSERT INTO dbprodutos.tb_produtos_2021_em_diante VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

In [28]:
cursor.executemany(sql, products_2021_onwards_list)
cnx.commit()

In [29]:
print(cursor.rowcount, "dados foram inseridos")

6574 dados foram inseridos


In [30]:
df_products_2021_onwards.shape

(6574, 13)

## Visualizando os dados inseridos

In [31]:
cursor.execute("SELECT * FROM dbprodutos.tb_livros")

for row in cursor:
    print(row)

('678fc8ec5028200ccb8bf14b', 'Modelagem preditiva', 'livros', 92.45, 5.61, datetime.date(2020, 1, 1), 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('678fc8ec5028200ccb8bf14c', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('678fc8ec5028200ccb8bf15e', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('678fc8ec5028200ccb8bf16c', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('678fc8ec5028200ccb8bf16e', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('678fc8ec5028200ccb8bf171', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credi

In [32]:
cursor.execute("SELECT * FROM dbprodutos.tb_produtos_2021_em_diante")

for row in cursor:
    print(row)

('678fc8ec5028200ccb8bf154', 'Xadrez de madeira', 'brinquedos', 25.23, 0.0, datetime.date(2021, 1, 1), 'Thiago Silva', 'BA', 5, 'cartao_credito', 2, -13.29, -41.71)
('678fc8ec5028200ccb8bf155', 'Impressora', 'eletronicos', 322.04, 14.73, datetime.date(2021, 1, 1), 'João Souza', 'SP', 3, 'cartao_credito', 1, -22.19, -48.79)
('678fc8ec5028200ccb8bf156', 'Mesa de centro', 'moveis', 282.22, 12.61, datetime.date(2021, 1, 1), 'João Souza', 'SP', 3, 'boleto', 1, -22.19, -48.79)
('678fc8ec5028200ccb8bf157', 'Tablet ABXY', 'eletronicos', 1100.42, 68.45, datetime.date(2021, 1, 1), 'Beatriz Moraes', 'SP', 1, 'cupom', 1, -22.19, -48.79)
('678fc8ec5028200ccb8bf158', 'Fogão', 'eletrodomesticos', 791.81, 42.44, datetime.date(2021, 1, 1), 'Juliana Costa', 'SP', 4, 'boleto', 1, -22.19, -48.79)
('678fc8ec5028200ccb8bf159', 'Violão', 'instrumentos musicais', 316.84, 17.16, datetime.date(2022, 1, 1), 'Pedro Gomes', 'SP', 5, 'cartao_credito', 2, -22.19, -48.79)
('678fc8ec5028200ccb8bf15a', 'Geladeira', 'el